In [1]:
import json
import numpy as np

from tsaug import TimeWarp, Crop, Quantize, Drift, Reverse
from tsaug.visualization import plot

In [2]:
with open("../parameters.json", "r") as f:
    para_dict = json.load(f)
    data_paths = para_dict["input_data_path"]  # dataset path

In [3]:
my_augmenter = (
    TimeWarp() #* 2  # random time warping 5 times in parallel
    # + Crop(size=300)  # random crop subsequences with length 300
    +
    Quantize(n_levels=[10, 20, 30])  # random quantize to 10-, 20-, or 30- level sets
    + 
    Drift(max_drift=(0.1, 0.5)) @ 0.8  # with 80% probability, random drift the signal up to 10% - 50%
    + 
    # 
    Reverse() @ 0.5  # with 50% probability, reverse the sequence
)

In [4]:
X = X_aug = []

for data_path in data_paths:
    x = np.genfromtxt('../' + data_path, delimiter=',', dtype=np.float32).transpose()
    from pathlib import Path
    pth = Path(data_path)
    fn = pth.with_suffix('').stem
    X_aug = my_augmenter.augment(x)
    
    np.savetxt("combined_" + fn + ".csv", X_aug.transpose(), delimiter=',')
    #plot(x, X_aug);
